In [1]:
import tensorflow as tf
import numpy as np

#作用：
# 1)make this notebook's output stble across runs;
# 2)清空defalt_graph
def reset_graph(seed = 318):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [17]:
data_path = "/data/stu12/mnist/"
data_path2 = "/data/stu12/mnist_2/"
# mnist = input_data.read_data_sets(data_path, one_hot=True)  # one_hot coding
mnist = input_data.read_data_sets(data_path2) # raw_coding

Extracting /data/stu12/mnist_2/train-images-idx3-ubyte.gz
Extracting /data/stu12/mnist_2/train-labels-idx1-ubyte.gz
Extracting /data/stu12/mnist_2/t10k-images-idx3-ubyte.gz
Extracting /data/stu12/mnist_2/t10k-labels-idx1-ubyte.gz


In [46]:
#图片常量
pic_height = 28
pic_width = 28
pic_size = pic_height*pic_width
pic_classes = 10
pic_channels = 1

#网络拓扑的常量
n_inputs = pic_size
n_hidden1 = 200
n_hidden2 = 120
n_outputs = pic_classes

#迭代次数
n_epochs = 21

# mini-batch
batch_size = 50
n_train_batches = mnist.train.num_examples // batch_size
n_test_batches = mnist.test.num_examples // batch_size

# learning_rate
learning_rate = 1e-4

# 构建模型

In [5]:
reset_graph()

### 定义网络的全连接层创建函数

In [6]:
# def neurons_layer(X, n_neurons, activation = None):
#     # 获取输入单元个数
#     n_inputs = int(X.get_shape()[-1])  # (batch_size, pic_size)
    
#     # 定义W
#     stddev = 2/np.sqrt(n_inputs)
#     init = tf.truncated_normal((n_inputs, n_neurons), stddev)
#     W = tf.Variable(init)
    
#     # 定义b
#     b = tf.Variable(tf.zeros((n_neurons), tf.float32))
    
#     # 加法
#     sigma = tf.matmul(X, W) + b
    
#     # 非线性变换
#     if activation is not None:
#         return activation(sigma)
    
#     # 不做非线性变换
#     return sigma    

### 构建神经网络

In [19]:
#定义placeholder (batch_size 《==》 None)
X = tf.placeholder(tf.float32, (None, pic_size))
# Y = tf.placeholder(tf.float32, (None, pic_classes))
Y = tf.placeholder(tf.int32, (None))

In [50]:
# hidden1 = neurons_layer(X, n_hidden1, tf.nn.relu) #  (batch_size, n_hidden1)
# hidden2 = neurons_layer(hidden1, n_hidden2, tf.nn.relu) # (batch_size, n_hidden2)
# prediction = neurons_layer(hidden2, n_outputs)   # (batch_size, n_outputs)
hidden1 = tf.layers.dense(X, n_hidden1, tf.nn.relu)
hidden1_dropout = tf.nn.dropout(hidden1, 0.7)
hidden2 = tf.layers.dense(hidden1_dropout, n_hidden2, tf.nn.relu)
hidden2_dropout = tf.nn.dropout(hidden2, 0.7)
prediction = tf.layers.dense(hidden2_dropout, n_outputs)

### 交叉熵损失(训练时用)

In [40]:
# xentropy = tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = prediction)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = Y, 
                                                         logits = prediction)
loss = tf.reduce_mean(xentropy)

### 定义学习算法

In [41]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### 准确率（测试、验证用，也可以在训练时）

In [42]:
# correct = tf.equal(tf.argmax(Y, 1), tf.argmax(prediction, 1)) # y.shape =(batch_size, pic_classes)
correct = tf.nn.in_top_k(prediction, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# 创建会话，进行训练、测试

In [43]:
#保存模型
saver = tf.train.Saver()
model_path = "/data/stu12/my_model/mlp.ckpt"

#模型

In [44]:
init = tf.global_variables_initializer()

In [52]:
with tf.Session() as sess:
    sess.run(init)
#     saver.restore(sess, model_path)
    
    # 进行多趟的训练和测试
    for epoch in range(n_epochs):
        
        # mini-batch
        train_acc = .0
        test_acc = .0
        for batch in range(n_train_batches):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            #fetch
            train_val, acc_val = sess.run([train, accuracy], 
                                 feed_dict = {X:x_batch, Y:y_batch})
            train_acc += acc_val
        train_acc /= n_train_batches
        
        for batch in range(n_test_batches):
            x_batch, y_batch = mnist.test.next_batch(batch_size)            
            acc_val = sess.run(accuracy, feed_dict = {X:x_batch, Y:y_batch})
            test_acc += acc_val
        test_acc /= n_test_batches
        
        print("Epoch " + str(epoch) +
             "; Train_acc:" + str(train_acc) +
             "; Test_acc:" + str(test_acc) + ";")

    saver.save(sess, model_path)

Epoch 0; Train_acc:0.856781811352; Test_acc:0.923999999762;
Epoch 1; Train_acc:0.932272726514; Test_acc:0.940700000823;
Epoch 2; Train_acc:0.946254546696; Test_acc:0.951400002241;
Epoch 3; Train_acc:0.956272728389; Test_acc:0.957300001383;
Epoch 4; Train_acc:0.963254547932; Test_acc:0.961000002027;
Epoch 5; Train_acc:0.968309092088; Test_acc:0.965000000894;
Epoch 6; Train_acc:0.972781817696; Test_acc:0.967300000489;
Epoch 7; Train_acc:0.975745454105; Test_acc:0.970100000799;
Epoch 8; Train_acc:0.978818182187; Test_acc:0.971699997485;
Epoch 9; Train_acc:0.980327273282; Test_acc:0.973299998641;
Epoch 10; Train_acc:0.98316363546; Test_acc:0.974100002348;
Epoch 11; Train_acc:0.984418181506; Test_acc:0.976100000739;
Epoch 12; Train_acc:0.98601818101; Test_acc:0.975699999928;
Epoch 13; Train_acc:0.987418181463; Test_acc:0.977100001574;
Epoch 14; Train_acc:0.988999999924; Test_acc:0.975999997854;
Epoch 15; Train_acc:0.990199999159; Test_acc:0.977099999189;
Epoch 16; Train_acc:0.991181817488; 